## Compute lag of multiple time series using Pandas


Let's look at an example where we have multiple time-series defined over different time periods (e.g., sales volumes for different countries, but the sales period may start at different times in different countries).



In [2]:
import pandas as pd
import numpy as np

#### We create a toy dataset for this example.


This representation means that multiple time-series at different time stamps are stored in the same columns `sales` and `date`.

In [20]:
time_us = pd.date_range(start='2000-01-01', end='2000-01-10')
y_us = np.random.randint(low=10, high=100, size=len(time_us))

time_uk = pd.date_range(start='2000-01-05', end='2000-01-15')
y_uk = np.random.randint(low=10, high=100, size=len(time_uk))


df_us = pd.DataFrame(data={'date':time_us, 'sales':y_us, 'country':'USA'}).set_index(keys=['date'])
df_uk = pd.DataFrame(data={'date':time_uk, 'sales':y_uk, 'country':'UK'}).set_index(keys=['date'])


df = pd.concat([df_us, df_uk])

df

,sales,country
date,,
2000-01-01,20,USA
2000-01-02,48,USA
2000-01-03,11,USA
2000-01-04,20,USA
2000-01-05,47,USA
2000-01-06,23,USA
2000-01-07,60,USA
2000-01-08,14,USA
2000-01-09,65,USA


### Let's look at potential mistakes in trying to use `shift` for this dataframe

### Suppose I want to create a lag of 2 


In [21]:
df.shift(periods=2)

,sales,country
date,,
2000-01-01,NaN,NaN
2000-01-02,NaN,NaN
2000-01-03,20.0,USA
2000-01-04,48.0,USA
2000-01-05,11.0,USA
2000-01-06,20.0,USA
2000-01-07,47.0,USA
2000-01-08,23.0,USA
2000-01-09,60.0,USA


### This is wrong as you can see the values from one timeseries have been shifted into the other.


### To get the correct result we want to set the index to be a datetime type and specify the frequency that we want to shift the index by and re-join to the original dataframe.

In [22]:
# Shift the timeseries index to get the lagged versions

df.shift(freq='5D')

,sales,country
date,,
2000-01-06,20,USA
2000-01-07,48,USA
2000-01-08,11,USA
2000-01-09,20,USA
2000-01-10,47,USA
2000-01-11,23,USA
2000-01-12,60,USA
2000-01-13,14,USA
2000-01-14,65,USA


In [24]:
assert all(df.shift(freq='5D') == df.shift( periods=5, freq='D'))

In [35]:
df_shift = df.shift(freq='5D')

df.merge(right=df_shift, suffixes=['', '_lag'], how='left', on=['date', 'country'])

,sales,country,sales_lag
date,,,
2000-01-01,20,USA,NaN
2000-01-02,48,USA,NaN
2000-01-03,11,USA,NaN
2000-01-04,20,USA,NaN
2000-01-05,47,USA,NaN
2000-01-06,23,USA,20.0
2000-01-07,60,USA,48.0
2000-01-08,14,USA,11.0
2000-01-09,65,USA,20.0
